In [ ]:
#%pip install -U llama-index llama-index-vector-stores-redis llama-index-embeddings-cohere llama-index-embeddings-openai
#%pip install llama-index-vector-stores-postgres
%pip install llama-index-storage-docstore-redis
%pip install llama-index-vector-stores-redis
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-openai

- https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from llama_index.core.schema import TextNode
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from sqlalchemy import make_url

import os
from llama_index.core import Settings, Document





In [41]:
from llama_index.core import SimpleDirectoryReader

# load documents with deterministic IDs
documents = SimpleDirectoryReader(
    "./nodestext", filename_as_id=True
).load_data()

In [54]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import (
    DocstoreStrategy,
    IngestionPipeline,
    IngestionCache,
)
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding

from redisvl.schema import IndexSchema
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = OpenAIEmbedding()
custom_schema = IndexSchema.from_dict(
    {
        "index": {"name": "redis_vector_store_openai4", "prefix": "doc"},
        # customize fields that are indexed
        "fields": [
            # required fields for llamaindex
            {"type": "tag", "name": "id"},
            {"type": "tag", "name": "doc_id"},
            {"type": "text", "name": "text"},
            {
                "type": "vector",
                "name": "vector",
                "attrs": {
                    "dims": 1536,
                    "algorithm": "hnsw",
                    "distance_metric": "cosine",
                },
            },
        ],
    }
)

In [56]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        embed_model,
    ],
    docstore=RedisDocumentStore.from_host_and_port(
        "localhost", 6379, namespace="document_store"
    ),
    vector_store=RedisVectorStore(
        schema=custom_schema,
        redis_url="redis://localhost:6379",
    ),
    #vector_store=pgvector_store,
    cache=IngestionCache(
        cache=RedisCache.from_host_and_port("localhost", 6379),
        collection="redis_cache",
    ),
    docstore_strategy=DocstoreStrategy.UPSERTS,
)

In [57]:
nodes = pipeline.run(documents=documents)
print(f"Ingested {len(nodes)} Nodes")

Ingested 1910 Nodes


In [58]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    pipeline.vector_store,  
    embed_model=embed_model
)

In [59]:
print(
    index.as_query_engine(similarity_top_k=10).query(
        "What documents do you see? and what categories"
    )
)

I see the following documents:
1. Document with Id: 189305 - Category: Catalog
2. Document with Id: 193147 - Category: Public Safety Key-chain
3. Document with Id: 194331 - Category: Video Intercom & Face Recognition Door Station
4. Document with Id: 194333 - Category: USB Card Issuer
5. Document with Id: 194319 - Category: Access Control - 4-Door Controller
6. Document with Id: 193719 - Category: Collector Pocket Knife
7. Document with Id: 188217 - Category: Evil Cat Keychain
8. Document with Id: 188220 - Category: Evil Cat Keychain
9. Document with Id: 188218 - Category: Evil Cat Keychain
10. Document with Id: 194316 - Category: Access Control - 2-Door Controller


In [60]:
print(
    index.as_chat_engine(similarity_top_k=10).chat(
        "tell me about knife"
    )
)

Knives come in various types such as fixed blade knives, spring-assisted knives, knuckle knives, and survival knives. They are made from high-quality materials like stainless steel and aluminum, offering durability and reliable performance. Knives serve different purposes including self-defense, outdoor activities, tactical operations, and everyday tasks. They feature sharp blades, ergonomic handles, pocket clips, and unique designs like scorpion or US flag motifs.
